In [ ]:
import sys

sys.path.append("..")

from src.auth.aurora import get_sales_client, get_portal_client


In [ ]:
sales_client = get_sales_client()
sales_hardware_df = sales_client.query_to_pandas(
    """SELECT
        h.device_id,
        s.subscription_type,
        s.start_date,
        s.expiry_date,
        h.unit_price,
        (
            CASE WHEN s.pay_frequency = "Monthly" THEN s.unit_price
            ELSE s.unit_price / s.contract_term
            END
        ) AS mrr
    FROM sales.t_hardware AS h
    LEFT JOIN sales.t_subscription AS s
    ON h.device_id = s.device_id
    WHERE h.status = "Dispatched"
    AND h.device_id IS NOT NULL
""",
)
sales_serials = sales_hardware_df["device_id"].unique().tolist()
sales_hardware_df.set_index("device_id", inplace=True)

In [ ]:
portal_client = get_portal_client()
placeholders = ', '.join(['%s'] * len(sales_serials))
portal_hardware_df = portal_client.query_to_pandas(
    f"""SELECT device_serial, client_id
    FROM topaz.api_devices AS d
    LEFT JOIN topaz.api_device_types AS t
    ON d.device_type_id = t.id
    WHERE active = 1
    AND device_serial IN ({placeholders})""",
    params=sales_serials
)
portal_hardware_df.set_index("device_serial", inplace=True)

In [30]:
full_hardware_df = sales_hardware_df.join(
    portal_hardware_df,
    how="inner",
)

In [35]:
import datetime

In [37]:
full_hardware_df["expiry_date"] >= datetime.date.today()

device_id
40972        False
40972        False
40990         True
40990        False
40990        False
             ...  
C01-00710     True
C01-00740    False
C01-00783    False
G01-01287     True
G01-01295     True
Name: expiry_date, Length: 4913, dtype: bool

In [17]:
root_client_df = portal_client.query_to_pandas(
    """WITH RECURSIVE category_path AS (
    SELECT id, name, parent_id, id AS root_id
    FROM api_clients
    WHERE parent_id = 1
    AND deleted_at IS NULL

    UNION ALL

    SELECT c.id, c.name, c.parent_id, cp.root_id AS root_id
    FROM api_clients c
    INNER JOIN category_path cp ON cp.id = c.parent_id
    AND deleted_at IS NULL
    )
    SELECT root_id, COUNT(DISTINCT user_id) AS user_cnt
    FROM category_path cp
    LEFT JOIN api_collaborators col
    ON cp.id = col.client_id
    WHERE col.deleted_at IS NULL
    GROUP BY root_id""",
)

In [18]:
root_client_df

,root_id,user_cnt
0,2,63
1,11,6
2,12,16
3,13,651
4,14,2
...,...,...
605,32220,0
606,32324,1
607,32463,6
608,32464,1
